In [1]:
import pandas as pd
from utils import get_path, group_cols
from datatable import dt, f, by, as_type

# Descrição

Neste notebook iremos formar a tabela procedimentos.

# Tabelas base

## Localidades

In [2]:
path_locs = get_path('LOCALIDADES', 'localidades.csv.gzip')
df_locs = dt.fread(path_locs)
df_locs

,codigo,cod_municipio,nome_municipio,capital,cod_regiao_saude,nome_regiao_saude,cod_uf,uf,nome_uf,sigla_regiao,nome_regiao,socioecon,latitude,longitude
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,CO/GO/52001/520005,520005,Abadia de Goiás,0,52001,Central,52,GO,Goiás,CO,Centro-Oeste,5,−16.7573,−49.4412
1,SD/MG/31052/310010,310010,Abadia dos Dourados,0,31052,Patrocínio Monte Carmelo,31,MG,Minas Gerais,SD,Sudeste,2,−18.4831,−47.3916
2,CO/GO/52011/520010,520010,Abadiânia,0,52011,Pirineus,52,GO,Goiás,CO,Centro-Oeste,3,−16.197,−48.7057
3,SD/MG/31067/310020,310020,Abaeté,0,31067,Sete Lagoas,31,MG,Minas Gerais,SD,Sudeste,3,−19.1551,−45.4444
4,NT/PA/15011/150010,150010,Abaetetuba,0,15011,Tocantins,15,PA,Pará,NT,Norte,1,−1.72183,−48.8788
5,ND/CE/23019/230010,230010,Abaiara,0,23019,19ª Região Brejo Santo,23,CE,Ceará,ND,Nordeste,1,−7.34588,−39.0416
6,ND/BA/29023/290010,290010,Abaíra,0,29023,Seabra,29,BA,Bahia,ND,Nordeste,1,−13.2488,−41.6619
7,ND/BA/29017/290020,290020,Abaré,0,29017,Paulo Afonso,29,BA,Bahia,ND,Nordeste,1,−8.72073,−39.1162
8,SL/PR/41018/410010,410010,Abatiá,0,41018,18ª RS Cornélio Procópio,41,PR,Paraná,SL,Sul,3,−23.3049,−50.3133


## Distâncias

In [3]:
cols_dist = {
  'origem': 'cod_municipio',
  'destino': 'hosp_cod_municipio',
  'distancia (km)': 'distancia (km)',
  'tempo (min)': 'tempo (min)',
  ...: None,
}

In [4]:
path_dist = get_path('DISTANCIAS', 'deslocamentos.csv.gzip')
df_dist = dt.fread(path_dist, columns=cols_dist)
df_dist

,cod_municipio,hosp_cod_municipio,distancia (km),tempo (min)
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,355030,0,0
1,330455,330455,0,0
2,130260,130260,0,0
3,530010,530010,0,0
4,230440,230440,0,0
5,292740,292740,0,0
6,310620,310620,0,0
7,150140,150140,0,0
8,431490,431490,0,0


## Rotas Gmaps

### Colunas

In [5]:
cols_gmaps = [
  'origem', 'destino', 'count',
  'distance (value)', 'duration (value)',
  'duration_in_traffic (value)',
]

### Carregando tabela

In [6]:
path_routes = get_path('GESTANTES', f'gmaps/routes.csv.gzip')
df_files = dt.fread(path_routes)
df_files

,status,origem,destino,origin_addresses,destination_addresses,origem_latitude,origem_longitude,destino_latitude,destino_longitude,hour,…,duration (value),duration (text),duration_in_traffic (value),duration_in_traffic (text),count
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,ZERO_RESULTS,292070,291360,"-14.1035,-39.0137","-14.793,-39.046",−14.1035,−39.0137,−14.793,−39.046,6,…,NA,,NA,NA,678
1,ZERO_RESULTS,290515,293330,"-14.3347,-40.9175","-14.8615,-40.8442",−14.3347,−40.9175,−14.8615,−40.8442,6,…,NA,,NA,NA,883
2,OK,330330,330490,"R. Maria Bregua, 37 - São Lourenço, Niterói - RJ, 24120-007, Brasil","R. Temistocles de Almeida, 134 - Camarão, São Gonçalo - RJ, 24436-820, Brasil",−22.8832,−43.1034,−22.8268,−43.0634,6,…,2271,38 minutos,NA,NA,204
3,OK,330330,330185,"R. Maria Bregua, 37 - São Lourenço, Niterói - RJ, 24120-007, Brasil","R. João da Silva Maia, 14 - Parque Santo Antonio, Guapimirim - RJ, 25945-348, Brasil",−22.8832,−43.1034,−22.5347,−42.9895,6,…,12388,3 horas 26 minutos,NA,NA,2
4,OK,330330,330023,"R. Maria Bregua, 37 - São Lourenço, Niterói - RJ, 24120-007, Brasil","Av. José Bento Ribeiro Dantas, 842, Búzios - RJ, 28950-000, Brasil",−22.8832,−43.1034,−22.7528,−41.8846,6,…,28870,8 horas 1 minuto,NA,NA,1
5,OK,330330,330170,"R. Maria Bregua, 37 - São Lourenço, Niterói - RJ, 24120-007, Brasil","R. Ana Neri, 18 - Jardim Vinte e Cinco de Agosto, Duque de Caxias - RJ, 25580-430, Brasil",−22.8832,−43.1034,−22.7858,−43.3049,6,…,4995,1 hora 23 minutos,NA,NA,25
6,ZERO_RESULTS,330330,330130,"-22.8832,-43.1034","-22.4812,-42.2066",−22.8832,−43.1034,−22.4812,−42.2066,6,…,NA,,NA,NA,2
7,ZERO_RESULTS,330330,330452,"-22.8832,-43.1034","-22.5174,-41.9475",−22.8832,−43.1034,−22.5174,−41.9475,6,…,NA,,NA,NA,3
8,OK,330330,330080,"R. Maria Bregua, 37 - São Lourenço, Niterói - RJ, 24120-007, Brasil","Rua Bernardo dos Santos 34 Sobrado - Centro, Cachoeiras de Macacu - RJ, 28680-000, Brasil",−22.8832,−43.1034,−22.4658,−42.6523,6,…,12930,3 horas 36 minutos,NA,NA,9


### Selecionando

driving pessimistic at 6pm

In [7]:
df_gmaps = df_files[
  (
    f.status == 'OK'
  ) & (
    f.mode == 'driving'
  ) & (
    f.traffic_model == 'pessimistic'
  ) & (
    f.hour == 18
  ), cols_gmaps]
df_gmaps

,origem,destino,count,distance (value),duration (value),duration_in_traffic (value)
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,354820,353800,91,32084,2515,2848
1,354820,350250,1,63291,4907,6096
2,354820,355480,5,27599,2101,2430
3,354820,354990,3,71554,3987,5074
4,354820,314730,1,46698,3317,3621
5,354820,354860,45,30568,2508,2668
6,354820,350970,448,21906,1835,2274
7,354820,355410,17,32628,2609,3203
8,120034,120040,153,225882,13228,14222


## SIH

### Colunas da tabela

In [8]:
cols_proc = [
  'ano',
  'parto_normal',
  'cod_municipio',
  'hosp_cod_municipio',
  'cnes',
]

In [9]:
path_sih = get_path('SIH', 'sih.jay')
df_proc = dt.fread(path_sih)
df_proc = df_proc[:, cols_proc]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,310010039,120005,120005,2001020
1,2010,310010039,120040,120040,2000733
2,2010,310010039,120040,120040,2000733
3,2010,310010039,120040,120040,2000733
4,2010,310010039,120040,120040,2000733
5,2010,310010039,120040,120040,2000733
6,2010,310010039,120040,120040,2000733
7,2010,310010039,120005,120005,2001020
8,2010,310010039,120040,120040,2000733


### Tabela agrupada

In [10]:
df_proc = group_cols(df_proc, cols_proc)
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,310010039,110001,110001,2679477,141
1,2010,310010039,110001,110002,2494299,1
2,2010,310010039,110001,110013,2808617,2
3,2010,310010039,110001,110020,4001303,2
4,2010,310010039,110001,110028,2495228,9
5,2010,310010039,110002,110002,2494299,507
6,2010,310010039,110002,110011,2808609,5
7,2010,310010039,110002,110012,2495279,1
8,2010,310010039,110002,110020,3970442,1


### Atualizando variável parto normal

In [11]:
# {310010039: 'normal', 411010034: 'cesariano'}
df_proc[:, dt.update(parto_normal = f.parto_normal == 310010039)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,1,110001,110001,2679477,141
1,2010,1,110001,110002,2494299,1
2,2010,1,110001,110013,2808617,2
3,2010,1,110001,110020,4001303,2
4,2010,1,110001,110028,2495228,9
5,2010,1,110002,110002,2494299,507
6,2010,1,110002,110011,2808609,5
7,2010,1,110002,110012,2495279,1
8,2010,1,110002,110020,3970442,1


### Ordenando linhas

In [12]:
df_proc = df_proc.sort(
  f.ano, f.parto_normal, f.cod_municipio, f.hosp_cod_municipio, f.cnes)
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49
1,2010,0,110001,110012,2495279,2
2,2010,0,110001,110013,2808617,1
3,2010,0,110001,110018,2496534,1
4,2010,0,110001,110028,2495228,4
5,2010,0,110001,351110,2089327,1
6,2010,0,110002,110002,2494299,375
7,2010,0,110002,110011,2808609,4
8,2010,0,110002,110020,4001303,1


# Gerando tabela

## Colunas origem & destino

In [13]:
def add_info_cols(df, cols, ref):
  df_codes = df[:, cols]
  col_key = cols[0]
  if ref == 'destino':
    col_key = 'hosp_' + col_key
    df_codes.names = {cols[0]: col_key}
  df_codes.names = {c: f'{ref}_{c}' for c in cols[2:]}
  df_codes.names = {cols[1]: ref}
  df_codes.key = col_key
  return df_codes

In [14]:
cols = ['cod_municipio', 'codigo', 'socioecon', 'capital', 'latitude', 'longitude']
df_origem = add_info_cols(df_locs, cols, 'origem')
df_destino = add_info_cols(df_locs, cols, 'destino')
df_proc = df_proc[:, :, dt.join(df_origem)]
df_proc = df_proc[:, :, dt.join(df_destino)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino,destino_socioecon,destino_capital,destino_latitude,destino_longitude
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110001,1,0,−11.9283,−61.9953
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11003/110012,2,0,−10.8777,−61.9322
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11001/110013,1,0,−9.44363,−61.9818
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11002/110018,3,0,−11.672,−61.198
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110028,1,0,−11.7271,−61.7714
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,SD/SP/35151/351110,3,0,−21.1314,−48.977
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11001/110002,1,0,−9.90571,−63.0325
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11003/110011,2,0,−10.4318,−62.4788
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11004/110020,3,1,−8.76077,−63.8999


## CNES concatenado ao destino

In [15]:
df_proc['cnes'] = df_proc[:, as_type(f.cnes, str)]
df_proc[:, dt.update(cnes= ('0000000' + f.cnes)[-7:])]
df_proc[:, dt.update(destino=f.destino + '/' + f.cnes)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino,destino_socioecon,destino_capital,destino_latitude,destino_longitude
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110001/2679477,1,0,−11.9283,−61.9953
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11003/110012/2495279,2,0,−10.8777,−61.9322
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11001/110013/2808617,1,0,−9.44363,−61.9818
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11002/110018/2496534,3,0,−11.672,−61.198
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110028/2495228,1,0,−11.7271,−61.7714
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,SD/SP/35151/351110/2089327,3,0,−21.1314,−48.977
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11001/110002/2494299,1,0,−9.90571,−63.0325
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11003/110011/2808609,2,0,−10.4318,−62.4788
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11004/110020/4001303,3,1,−8.76077,−63.8999


## Colunas de distância e tempo

### OSM

In [16]:
df_dist.key = ['cod_municipio', 'hosp_cod_municipio']
df_proc = df_proc[:, :, dt.join(df_dist)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino_capital,destino_latitude,destino_longitude,distancia (km),tempo (min)
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,0,−11.9283,−61.9953,0,0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,0,−10.8777,−61.9322,130.77,152.868
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,0,−9.44363,−61.9818,357.511,427.84
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,0,−11.672,−61.198,114.061,112.283
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,0,−11.7271,−61.7714,48.6099,50.14
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,0,−21.1314,−48.977,2196.35,1852.41
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,0,−9.90571,−63.0325,0,0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,0,−10.4318,−62.4788,95.4727,140.33
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,1,−8.76077,−63.8999,203.582,188.315


### Gmaps

In [17]:
# df_gmaps.key = ['origem', 'destino']
# df_proc = df_proc[:, :, dt.join(df_dist)]
# df_proc

### Distância e tempo ponderado

In [18]:
# df_proc[:, dt.update(dist_pond = f['distancia (km)'] * f.count)]
# df_proc[:, dt.update(tempo_pond = f['tempo (min)'] * f.count)]
# df_proc

## Coluna socioecon

In [19]:
df_proc['origem_socioecon'] = df_proc[:, as_type(f.origem_socioecon, str)]
df_proc['destino_socioecon'] = df_proc[:, as_type(f.destino_socioecon, str)]
df_proc[:, dt.update(socioecon = f.origem_socioecon + '_' + f.destino_socioecon)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino_latitude,destino_longitude,distancia (km),tempo (min),socioecon
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,−11.9283,−61.9953,0,0,1_1
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,−10.8777,−61.9322,130.77,152.868,1_2
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,−9.44363,−61.9818,357.511,427.84,1_1
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,−11.672,−61.198,114.061,112.283,1_3
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,−11.7271,−61.7714,48.6099,50.14,1_1
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,−21.1314,−48.977,2196.35,1852.41,1_3
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,−9.90571,−63.0325,0,0,1_1
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,−10.4318,−62.4788,95.4727,140.33,1_2
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,−8.76077,−63.8999,203.582,188.315,1_3


## Coluna capitais

In [20]:
df_proc['origem_capital'] = df_proc[:, as_type(f.origem_capital, int)]
df_proc['origem_capital'] = df_proc[:, as_type(f.origem_capital, str)]
df_proc['destino_capital'] = df_proc[:, as_type(f.destino_capital, int)]
df_proc['destino_capital'] = df_proc[:, as_type(f.destino_capital, str)]
df_proc[:, dt.update(capitais = f.origem_capital + '_' + f.destino_capital)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino_longitude,distancia (km),tempo (min),socioecon,capitais
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,−61.9953,0,0,1_1,0_0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,−61.9322,130.77,152.868,1_2,0_0
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,−61.9818,357.511,427.84,1_1,0_0
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,−61.198,114.061,112.283,1_3,0_0
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,−61.7714,48.6099,50.14,1_1,0_0
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,−48.977,2196.35,1852.41,1_3,0_0
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,−63.0325,0,0,1_1,0_0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,−62.4788,95.4727,140.33,1_2,0_0
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,−63.8999,203.582,188.315,1_3,0_1


## Coluna criticidade

In [21]:
crits = list()
origins = df_proc[:, 'origem'].to_list()[0]
destins = df_proc[:, 'destino'].to_list()[0]
for ori, des in zip(origins, destins):
  critic = 0
  trim = ori
  while trim not in des:
    critic += 1
    trim = '/'.join(trim.split('/')[:-1])
  crits.append(critic)
df_proc['criticidade'] = dt.Frame(crits)
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,distancia (km),tempo (min),socioecon,capitais,criticidade
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,0,0,1_1,0_0,0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,130.77,152.868,1_2,0_0,2
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,357.511,427.84,1_1,0_0,2
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,114.061,112.283,1_3,0_0,2
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,48.6099,50.14,1_1,0_0,1
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,2196.35,1852.41,1_3,0_0,4
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,0,0,1_1,0_0,0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,95.4727,140.33,1_2,0_0,2
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,203.582,188.315,1_3,0_1,2


## Ajustando colunas

In [22]:
cols_order = [
  'ano',
  'parto_normal',
  'origem',
  'destino',
  'distancia (km)',
  'tempo (min)',
  # 'dist_pond',
  # 'tempo_pond',
  'criticidade',
  'capitais',
  'socioecon',
  'origem_latitude',
  'origem_longitude',
  'destino_latitude',
  'destino_longitude',
  'count',
]

In [23]:
df_proc = df_proc[:, cols_order]
df_proc

,ano,parto_normal,origem,destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0_0,1_1,−11.9283,−61.9953,−11.9283,−61.9953,49
1,2010,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,2,0_0,1_2,−11.9283,−61.9953,−10.8777,−61.9322,2
2,2010,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,2,0_0,1_1,−11.9283,−61.9953,−9.44363,−61.9818,1
3,2010,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,2,0_0,1_3,−11.9283,−61.9953,−11.672,−61.198,1
4,2010,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,1,0_0,1_1,−11.9283,−61.9953,−11.7271,−61.7714,4
5,2010,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,4,0_0,1_3,−11.9283,−61.9953,−21.1314,−48.977,1
6,2010,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0_0,1_1,−9.90571,−63.0325,−9.90571,−63.0325,375
7,2010,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,2,0_0,1_2,−9.90571,−63.0325,−10.4318,−62.4788,4
8,2010,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,2,0_1,1_3,−9.90571,−63.0325,−8.76077,−63.8999,1


# Biênios (2010/11 & 2018/19)

## Filtrando os anos

In [24]:
df_proc_bienio = df_proc[
  (f.ano == 2010) | (f.ano == 2011) | (f.ano == 2018) |(f.ano == 2019)
, :]
df_proc_bienio

,ano,parto_normal,origem,destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0_0,1_1,−11.9283,−61.9953,−11.9283,−61.9953,49
1,2010,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,2,0_0,1_2,−11.9283,−61.9953,−10.8777,−61.9322,2
2,2010,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,2,0_0,1_1,−11.9283,−61.9953,−9.44363,−61.9818,1
3,2010,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,2,0_0,1_3,−11.9283,−61.9953,−11.672,−61.198,1
4,2010,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,1,0_0,1_1,−11.9283,−61.9953,−11.7271,−61.7714,4
5,2010,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,4,0_0,1_3,−11.9283,−61.9953,−21.1314,−48.977,1
6,2010,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0_0,1_1,−9.90571,−63.0325,−9.90571,−63.0325,375
7,2010,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,2,0_0,1_2,−9.90571,−63.0325,−10.4318,−62.4788,4
8,2010,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,2,0_1,1_3,−9.90571,−63.0325,−8.76077,−63.8999,1


## Atualizando variável ano (biênio)

Valor da variável `bienio`:

- `0 (False) = 2010/11`
- `1 (True) = 2018/19`

In [25]:
df_proc_bienio[:, dt.update(ano = (f.ano == 2018) | (f.ano == 2019))]
df_proc_bienio.names = {'ano': 'bienio'}
df_proc_bienio

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0_0,1_1,−11.9283,−61.9953,−11.9283,−61.9953,49
1,0,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,2,0_0,1_2,−11.9283,−61.9953,−10.8777,−61.9322,2
2,0,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,2,0_0,1_1,−11.9283,−61.9953,−9.44363,−61.9818,1
3,0,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,2,0_0,1_3,−11.9283,−61.9953,−11.672,−61.198,1
4,0,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,1,0_0,1_1,−11.9283,−61.9953,−11.7271,−61.7714,4
5,0,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,4,0_0,1_3,−11.9283,−61.9953,−21.1314,−48.977,1
6,0,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0_0,1_1,−9.90571,−63.0325,−9.90571,−63.0325,375
7,0,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,2,0_0,1_2,−9.90571,−63.0325,−10.4318,−62.4788,4
8,0,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,2,0_1,1_3,−9.90571,−63.0325,−8.76077,−63.8999,1


### Tabela agrupada

In [26]:
df_proc_bienio = df_proc_bienio[:, dt.sum(f.count), by(df_proc_bienio.names[:-1])]
df_proc_bienio

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,964
5,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010510,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,832
6,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010529,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,1831
7,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010537,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,3707
8,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010545,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,728


# Exportando tabela

## Ano

In [27]:
output_proc = get_path('GESTANTES', 'procs_ano.csv.gzip')
df_proc.to_csv(output_proc)

In [28]:
dt.fread(output_proc)

,ano,parto_normal,origem,destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2010,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0_0,1_1,−11.9283,−61.9953,−11.9283,−61.9953,49
1,2010,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,2,0_0,1_2,−11.9283,−61.9953,−10.8777,−61.9322,2
2,2010,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,2,0_0,1_1,−11.9283,−61.9953,−9.44363,−61.9818,1
3,2010,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,2,0_0,1_3,−11.9283,−61.9953,−11.672,−61.198,1
4,2010,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,1,0_0,1_1,−11.9283,−61.9953,−11.7271,−61.7714,4
5,2010,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,4,0_0,1_3,−11.9283,−61.9953,−21.1314,−48.977,1
6,2010,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0_0,1_1,−9.90571,−63.0325,−9.90571,−63.0325,375
7,2010,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,2,0_0,1_2,−9.90571,−63.0325,−10.4318,−62.4788,4
8,2010,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,2,0_1,1_3,−9.90571,−63.0325,−8.76077,−63.8999,1


## Biênio

In [29]:
output_proc_bienio = get_path('GESTANTES', 'procs_bienio.csv.gzip')
df_proc_bienio.to_csv(output_proc_bienio)

In [30]:
dt.fread(output_proc_bienio)

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,964
5,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010510,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,832
6,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010529,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,1831
7,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010537,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,3707
8,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010545,0,0,0,1_1,5_5,−15.7795,−47.9297,−15.7795,−47.9297,728


# Unindo com Gmaps

## Fontes de dados

### Procedimentos

In [31]:
df_bienio = pd.read_csv(output_proc_bienio)
df_bienio = df_bienio.rename(columns={
  ref: f'codigo_{ref}' for ref in ['origem', 'destino']})
df_bienio['origem'] = df_bienio['codigo_origem'].apply(lambda x: int(x.split('/')[-1]))
df_bienio['destino'] = df_bienio['codigo_destino'].apply(lambda x: int(x.split('/')[-2]))
df_bienio

,bienio,parto_normal,codigo_origem,codigo_destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count,origem,destino
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,260,530010,530010
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,282,530010,530010
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3323,530010,530010
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3368,530010,530010
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,964,530010,530010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124980,1,1,SL/SC/42016/421870,SL/SC/42007/420540/3157245,138.4773,104.940000,2,0_1,3_5,-28.4713,-49.0144,-27.5945,-48.5477,1,421870,420540
124981,1,1,SL/SC/42016/421870,SL/SC/42015/420460/2594277,54.6210,49.316667,2,0_0,3_4,-28.4713,-49.0144,-28.6723,-49.3729,1,421870,420460
124982,1,1,SL/SC/42016/421870,SL/SC/42016/420280/2665883,35.5439,37.493333,1,0_0,3_3,-28.4713,-49.0144,-28.2681,-49.1701,2,421870,420280
124983,1,1,SL/SC/42016/421870,SL/SC/42016/420730/2385880,53.9610,45.353333,1,0_0,3_3,-28.4713,-49.0144,-28.2284,-48.6659,1,421870,420730


#### Biênio RJ

In [32]:
origem_rj = df_bienio['origem'].astype(str).str.startswith('33')
destino_rj = df_bienio['destino'].astype(str).str.startswith('33')
df_bienio_rj = df_bienio[origem_rj & destino_rj]
df_bienio_rj

,bienio,parto_normal,codigo_origem,codigo_destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count,origem,destino
20117,0,0,SD/RJ/33001/330010,SD/RJ/33001/330010/2280868,0.0000,0.000000,0,0_0,4_4,-23.0011,-44.3196,-23.0011,-44.3196,533,330010,330010
20118,0,0,SD/RJ/33001/330010,SD/RJ/33001/330010/2281384,0.0000,0.000000,0,0_0,4_4,-23.0011,-44.3196,-23.0011,-44.3196,116,330010,330010
20120,0,0,SD/RJ/33001/330260,SD/RJ/33001/330010/2280868,56.7885,47.100000,1,0_0,4_4,-22.9594,-44.0409,-23.0011,-44.3196,4,330260,330010
20121,0,0,SD/RJ/33001/330260,SD/RJ/33001/330260/2288109,0.0000,0.000000,0,0_0,4_4,-22.9594,-44.0409,-22.9594,-44.0409,183,330260,330260
20122,0,0,SD/RJ/33001/330260,SD/RJ/33005/330045/2289709,84.1763,63.915000,2,0_0,4_5,-22.9594,-44.0409,-22.7640,-43.3992,6,330260,330045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116597,1,1,SD/RJ/33009/330590,SD/RJ/33006/330330/0012521,186.0775,184.508333,2,0_0,3_5,-22.0638,-42.0643,-22.8832,-43.1034,1,330590,330330
116598,1,1,SD/RJ/33009/330590,SD/RJ/33008/330240/5412447,80.5919,75.171667,2,0_0,3_5,-22.0638,-42.0643,-22.3768,-41.7848,1,330590,330240
116599,1,1,SD/RJ/33009/330590,SD/RJ/33008/330415/2267209,79.7153,75.683333,2,0_0,3_5,-22.0638,-42.0643,-22.1031,-41.4693,1,330590,330415
116600,1,1,SD/RJ/33009/330590,SD/RJ/33009/330340/2271826,81.5882,79.888333,1,0_0,3_3,-22.0638,-42.0643,-22.2932,-42.5377,1,330590,330340


### Rotas (Gmaps)

In [33]:
cols_gmaps = [
  'origem', 'destino',
  # 'distancia (km)', 'tempo (min)',
  'distance (value)',
  'duration (value)', 'duration_in_traffic (value)'
]

In [34]:
path_route = get_path('GESTANTES', f'routes.csv.gzip')
df_route = pd.read_csv(path_route)
df_route = df_route[cols_gmaps]
df_route

,origem,destino,distance (value),duration (value),duration_in_traffic (value)
0,510840,510340,7.555,16.333333,34.016667
1,315460,310620,42.332,53.716667,82.650000
2,260790,261160,21.010,46.983333,77.250000
3,521250,530010,60.888,62.600000,83.350000
4,280480,280030,16.221,23.133333,31.483333
...,...,...,...,...,...
57271,520465,520110,409.137,329.583333,362.450000
57272,520470,170950,308.501,248.433333,265.916667
57273,520470,172100,525.439,422.766667,446.300000
57274,520470,520620,423.046,337.766667,385.066667


## Gerando tabela

### Combinando tabelas

In [35]:
df_proc_gmaps = pd.merge(df_bienio, df_route, how='left', on=['origem', 'destino'])
df_proc_gmaps

,bienio,parto_normal,codigo_origem,codigo_destino,distancia (km),tempo (min),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count,origem,destino,distance (value),duration (value),duration_in_traffic (value)
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,260,530010,530010,NaN,NaN,NaN
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,282,530010,530010,NaN,NaN,NaN
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3323,530010,530010,NaN,NaN,NaN
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,3368,530010,530010,NaN,NaN,NaN
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0.0000,0.000000,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,964,530010,530010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124980,1,1,SL/SC/42016/421870,SL/SC/42007/420540/3157245,138.4773,104.940000,2,0_1,3_5,-28.4713,-49.0144,-27.5945,-48.5477,1,421870,420540,136.964,103.016667,134.716667
124981,1,1,SL/SC/42016/421870,SL/SC/42015/420460/2594277,54.6210,49.316667,2,0_0,3_4,-28.4713,-49.0144,-28.6723,-49.3729,1,421870,420460,65.627,54.633333,75.150000
124982,1,1,SL/SC/42016/421870,SL/SC/42016/420280/2665883,35.5439,37.493333,1,0_0,3_3,-28.4713,-49.0144,-28.2681,-49.1701,2,421870,420280,35.550,43.833333,55.416667
124983,1,1,SL/SC/42016/421870,SL/SC/42016/420730/2385880,53.9610,45.353333,1,0_0,3_3,-28.4713,-49.0144,-28.2284,-48.6659,1,421870,420730,52.210,44.033333,59.200000


### Ordenando

#### Colunas

In [36]:
cols_order_gmaps = [
  'bienio', 'parto_normal',
  'codigo_origem', 'codigo_destino', 'origem', 'destino',
  # 'mode', 'traffic_model', 'hour',
  'distancia (km)', 'tempo (min)',
  'distance (value)',
  'duration (value)', 'duration_in_traffic (value)',
  # 'dist_pond', 'tempo_pond',
  'criticidade', 'capitais', 'socioecon',
  'origem_latitude', 'origem_longitude',
  'destino_latitude', 'destino_longitude',
  'count',
]

In [37]:
cols_rename = {
  'distancia (km)': 'distance (osm)',
  'tempo (min)': 'duration (osm)',
  'distance (value)': 'distance (gmaps)',
  'duration (value)': 'duration (gmaps)',
  'duration_in_traffic (value)': 'duration_traffic (gmaps)',
}

#### Linhas

In [38]:
df_proc_gmaps = df_proc_gmaps[cols_order_gmaps]
df_proc_gmaps = df_proc_gmaps.rename(columns=cols_rename)
df_proc_gmaps = df_proc_gmaps.sort_values([
  'codigo_origem', 'codigo_destino'])
df_proc_gmaps

,bienio,parto_normal,codigo_origem,codigo_destino,origem,destino,distance (osm),duration (osm),distance (gmaps),duration (gmaps),duration_traffic (gmaps),criticidade,capitais,socioecon,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,530010,530010,0.0000,0.000000,NaN,NaN,NaN,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,260
28243,0,1,CO/DF/53001/530010,CO/DF/53001/530010/0010464,530010,530010,0.0000,0.000000,NaN,NaN,NaN,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,2171
59725,1,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,530010,530010,0.0000,0.000000,NaN,NaN,NaN,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,1954
90381,1,1,CO/DF/53001/530010,CO/DF/53001/530010/0010464,530010,530010,0.0000,0.000000,NaN,NaN,NaN,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,2521
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,530010,530010,0.0000,0.000000,NaN,NaN,NaN,0,1_1,5_5,-15.7795,-47.9297,-15.7795,-47.9297,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90379,1,0,SL/SC/42016/421870,SL/SC/42016/420940/2558017,421870,420940,31.5939,26.976667,31.558,30.1,41.1,1,0_0,3_3,-28.4713,-49.0144,-28.4843,-48.7772,1
28242,0,0,SL/SC/42016/421870,SL/SC/42016/421870/2491710,421870,421870,0.0000,0.000000,NaN,NaN,NaN,0,0_0,3_3,-28.4713,-49.0144,-28.4713,-49.0144,52
59724,0,1,SL/SC/42016/421870,SL/SC/42016/421870/2491710,421870,421870,0.0000,0.000000,NaN,NaN,NaN,0,0_0,3_3,-28.4713,-49.0144,-28.4713,-49.0144,271
90380,1,0,SL/SC/42016/421870,SL/SC/42016/421870/2491710,421870,421870,0.0000,0.000000,NaN,NaN,NaN,0,0_0,3_3,-28.4713,-49.0144,-28.4713,-49.0144,190


## Exportando tabela resultante

In [39]:
path_proc_gmaps = get_path('GESTANTES', f'procs.csv.gzip')
df_proc_gmaps.to_csv(path_proc_gmaps, index=False)

In [40]:
# path_routes_rj = get_path('GESTANTES', f'gmaps/routes_rj.xlsx')
# df_proc_gmaps.to_excel(path_routes_rj, index=False)